In [1]:
import json, pycm, pandas as pd
from nl_classifier import NaturalLanguageClassifer
from datetime import datetime

In [2]:
MODEL_NAME = 'gpt-4'
TEMPERATURE = 0.1
RESULTS_FILENAME = 'woman_gpt-4_description_results.json'

In [3]:
experiments = json.load(open(RESULTS_FILENAME, 'r'))

In [5]:
for i, experiment in enumerate(experiments):
    cls = experiment["concept"]
    classifier = NaturalLanguageClassifer(cls["id"], cls["label"], cls["definition"], MODEL_NAME, TEMPERATURE)
    print(f'{i+1:02}: {cls["label"]:30} {cls["id"]}')
    data = experiment["data"]
    for j, entity in enumerate(data):
        if "predicted" in entity:
            print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} SKIPPING')
        else:
            print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} {entity["actual"]:10} {classifier.tokens_used(entity["label"] + " " + entity["description"]):05} tokens ', end=" ")
            classification = classifier.classify(entity["label"], entity["description"])
            entity["predicted"] = classification["predicted"].lower()
            if entity["actual"] != entity["predicted"]:
                if entity["actual"] == "positive":
                    print("FN")
                else:
                    print("FP")
            else:
                print("")
            entity["rationale"] = classification["rationale"]
        df_results = pd.DataFrame.from_records(data)
        cm = pycm.ConfusionMatrix(df_results["actual"].tolist(), df_results["predicted"].tolist(), digit=2, classes=[ 'positive', 'negative' ])
        experiment["confusion_matrix"] = cm.matrix
        experiment["created"] = datetime.now().isoformat()
        json.dump(experiments, open(RESULTS_FILENAME, 'w+'))

01: woman                          oed_woman
   01: Sally Pearson                  http://www.wikidata.org/entity/Q6026          SKIPPING
   02: Lynne Jolitz                   http://www.wikidata.org/entity/Q11742         SKIPPING
   03: Victoria Arellano              http://www.wikidata.org/entity/Q7926558       SKIPPING
   04: Jaqueline Jesus                http://www.wikidata.org/entity/Q9010833       SKIPPING
   05: Angela Merkel                  http://www.wikidata.org/entity/Q567           SKIPPING
   06: Miles Franklin                 http://www.wikidata.org/entity/Q984           SKIPPING
   07: Terri O'Connell                http://www.wikidata.org/entity/Q16195718      SKIPPING
   08: Esdras Parra                   http://www.wikidata.org/entity/Q5488684       SKIPPING
   09: Elke Mackenzie                 http://www.wikidata.org/entity/Q16003604      SKIPPING
   10: Manuela Trasobares             http://www.wikidata.org/entity/Q2469656       SKIPPING
   11: Chen Lili (model) 